In [20]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from file_exchange import config
from file_exchange.network import Network
from file_exchange.peers_view import PeersView
from file_exchange.user import User
import gevent
from gevent.pool import Group
from gevent.event import AsyncResult

from datetime import datetime

# Parallel users

In [21]:
conf = config.default
conf['n_users'] = 10
conf

{'chunk_max_size': 2,
 'file_size': 11,
 'gossip_size': 20,
 'layer_threshold': 0.001,
 'minimum_node_availability': 0,
 'n_file_chunks': 10,
 'n_layers': 3,
 'n_rounds': 2,
 'n_users': 10,
 'period': datetime.timedelta(0, 1)}

In [24]:
address_book = PeersView(conf)
net = Network()
users = [None] * conf['n_users']
for i in range(conf['n_users']):
    users[i] = User(address_book, net, conf)

def output_view(view, conf):
    i = 0
    while True:
        print("Iteration #{} at time {}".format(i, datetime.now().time()))
        print(view.get())
        gevent.sleep(conf['period'].total_seconds())
        i+=1

print("Before spawn")
output_thread = gevent.spawn(output_view, address_book, conf)
print("Before start")

for u in users:
    # Starts greenlet
    u.start()
gevent.joinall(users)

print("User thread joined")
gevent.kill(output_thread)


Before spawn
Before start
Iteration #0 at time 21:18:47.013546
Empty DataFrame
Columns: [t, addr, type, p]
Index: []
Iteration #1 at time 21:18:48.015768
                                    addr         p                          t  \
2018-04-24 21:18:47.018936  nuvtvipej9cg  0.517770 2018-04-24 21:18:47.018936   
2018-04-24 21:18:47.021471  dmqeda2q72u5  0.607986 2018-04-24 21:18:47.021471   
2018-04-24 21:18:47.023885  g5w19obk63va  0.931181 2018-04-24 21:18:47.023885   
2018-04-24 21:18:47.026308  i9qzpgb5akqm  0.632094 2018-04-24 21:18:47.026308   
2018-04-24 21:18:47.028876  bk0j0dwmc9lw  0.902881 2018-04-24 21:18:47.028876   
2018-04-24 21:18:47.031519  0kbewcnhitrj  0.836801 2018-04-24 21:18:47.031519   
2018-04-24 21:18:47.034129  r2xelwee8yd5  0.893604 2018-04-24 21:18:47.034129   
2018-04-24 21:18:47.036582  lcv60e5r8mp7  0.553732 2018-04-24 21:18:47.036582   
2018-04-24 21:18:47.039095  mpfxfa00uaor  0.846358 2018-04-24 21:18:47.039095   
2018-04-24 21:18:47.041581  g9q17ujb

# File sharing initialization

In [26]:
conf = config.default
conf['n_users'] = 10
print(conf)

address_book = PeersView(conf)
net = Network()
users = [None] * conf['n_users']
for i in range(conf['n_users']):
    users[i] = User(address_book, net, conf)
    users[i].update_state()
    
u1 = users[0]
u2 = users[1]

print("File sender named {} has following devices: {}".format(u1.name, [d.addr for d in u1.devices]))
print("File receiver named {} has following devices: {}".format(u2.name, [d.addr for d in u2.devices]))
u2.init_file_receive(u1)

{'n_file_chunks': 10, 'n_layers': 3, 'gossip_size': 20, 'n_users': 10, 'layer_threshold': 0.001, 'period': datetime.timedelta(0, 1), 'minimum_node_availability': 0, 'file_size': 11, 'n_rounds': 2, 'chunk_max_size': 2}
File sender named Georgene has following devices: ['ltj43szc4vfu', 'dfcn6i5gaebr', 'yk0nihc6jwgx', 'iydcaj3wqtvr', 'g9cr9y37iwns', 'rqd7y4ozgy3h', '76h9qi9imwri', 'hwm8mwwxxjl1', 'l3tm8s4zx7dd', 'yon56r90qde7']
File receiver named Deneen has following devices: ['repjprccy6i6', '2vw0z5cblwcp', 'qrq5mi4mapiq', '1f00aqu2pnxu', '5x312wlrckcm', '0p14ied9ouyc']


{'backward_route': [['rticn11f57q8',
   '4l0fud3fbgnx',
   'rqd7y4ozgy3h',
   'qrq5mi4mapiq',
   'eq07380e1vt5'],
  ['sx1qpqlokw7u', 'bkzsgui7quyg', 'sysr8h6jlxnl', '5x312wlrckcm'],
  ['565rrzp4pbys', 'hc6cagrnnnhy', 'bwmuh87bw41p', '5ak4wjmvhn8e'],
  ['ltj43szc4vfu',
   'dfcn6i5gaebr',
   'yk0nihc6jwgx',
   'iydcaj3wqtvr',
   'g9cr9y37iwns',
   'rqd7y4ozgy3h',
   '76h9qi9imwri',
   'hwm8mwwxxjl1',
   'l3tm8s4zx7dd',
   'yon56r90qde7']],
 'file': <file_exchange.file.File at 0x7fb83c96fdd8>,
 'forward_route': [['565rrzp4pbys',
   'hc6cagrnnnhy',
   'bwmuh87bw41p',
   '5ak4wjmvhn8e'],
  ['sx1qpqlokw7u', 'bkzsgui7quyg', 'sysr8h6jlxnl', '5x312wlrckcm'],
  ['rticn11f57q8',
   '4l0fud3fbgnx',
   'rqd7y4ozgy3h',
   'qrq5mi4mapiq',
   'eq07380e1vt5'],
  ['repjprccy6i6',
   '2vw0z5cblwcp',
   'qrq5mi4mapiq',
   '1f00aqu2pnxu',
   '5x312wlrckcm',
   '0p14ied9ouyc']]}

In [33]:
class A():
    def __init__(self, x):
        self.x = x
        
a = {'x': A(5), 'b' : 3}

def lol(a):
    a['x'].x = 3
    
lol(a)
a['x'].x

3

In [35]:
for k, v in a.items():
    print(k,v)

x <__main__.A object at 0x7fb83c81d2e8>
b 3
